# Step 1

* Load scalar, waveform and image h5 files
* Snap points onto a grid in x1-x2 space.
* Interpolate onto regular y1 grid.
* Generate file "rawgrid_..." containing the 5D density array in slit-screen (x1-x2-y1-x3-y3) coordinates.
* Generate file "slit_coordinates_..." containing the coordinates along each dimension in "rawgrid...".

In [ ]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import numpy as np
import pandas as pd
import h5py
import imageio
from scipy import ndimage
from scipy import interpolate
import skimage
from tqdm import tqdm
from tqdm import trange
from matplotlib import pyplot as plt
import proplot as pplt

sys.path.append('../..')
from tools import energyVS06 as energy
from tools import image_processing as ip
from tools import plotting as mplt
from tools import utils

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

## Setup 

Global variables

In [ ]:
N_PTS_Y = 33  # Number of points in y grid (sweeping slit).
SMOOTH = False  # Whether to smooth the 3D (y1-y3-x3) image along the first axis.
ny, nx = np.loadtxt('processed_image_shape.txt').astype('int')
print(f'Processed image shape = ({ny}, {nx})')

In [ ]:
datadir = '/Diagnostics/Data/Measurements/scan-xxpy-image-ypdE/2022-04-29/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220429190854-scan-xxpy-image-ypdE'
file = h5py.File(join(datadir, 'preproc-' + filename + '.h5'), 'r')
data_sc = file['/scalardata']
data_wf = file['/wfdata']
data_im = file['/imagedata']

print('All attributes:')
print()
for data in [data_sc, data_wf, data_im]:
    print(data.name)
    for item in data.dtype.fields.items():
        print(item)
    print()

In [ ]:
cam = 'cam34'
sdiag = ['bcm04', f'{cam}_Integral', f'{cam}_Saturation']
acts = ['y_PositionSync', 'xp_PositionSync', 'x_PositionSync']
print('Scalar diagnostics:', sdiag)
print('Actuators:', acts)

In [ ]:
signal = np.copy(data_sc[:, cam + '_Integral'])
ipeak = np.argmax(signal)
print('Index of peak signal:', ipeak)

### Slit correlations

In [ ]:
corr = np.identity(len(acts))
intercept = np.zeros(corr.shape)
for i, act1 in enumerate(acts):
    for j, act2 in enumerate(acts):
        fit1d = np.polyfit(data_sc[:, act1], data_sc[:, act2], 1, w=signal)
        print(f'{act2} vs. {act1}: slope = {fit1d[0]:.3f}')
        corr[i, j] = fit1d[0]
        intercept[i, j] = fit1d[1]
print('Correlation matrix:')
print(corr)

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=3, figwidth=4.0, 
                          spanx=False, spany=False, aligny=True)
for i in range(3):
    for j in range(3):
        ax = axes[i, j]
        x = data_sc[acts[j]]
        y = data_sc[acts[i]]
        ax.plot(x, y, color='black', alpha=0.3)
        xmin, xmax = np.min(x), np.max(x)
        _x = x - 0.5 * (xmax - xmin)
        _y = corr[j, i] * _x + intercept[j, i]
        ax.plot(x, _y, color='black')
for i, act in enumerate(acts):
    axes[-1, i].set_xlabel(act)
    axes[i, 0].set_ylabel(act)
plt.show()

Work in the "normalized" (decorrelated) slit-screen coordinates. First, try transforming back to square grid.

In [ ]:
# Shearing matrix 
ndim = 3
M = np.identity(ndim)
M[1, 2] = 1.9  # dxp / dx
Minv = np.linalg.inv(M)

# Center slit positions
d1_center = 6.5  # y
d2_center = 15.0  # xp
d3_center = 21.0  # x

center = np.array([d1_center, d2_center, d3_center])

Test on CSV file (planned points).

In [ ]:
# Find out how many rows to skip. 
csv_filename = join(datadir, filename + '.csv')
skiprows = 0
for line in open(csv_filename, 'r'):
    if line.startswith('#'):
        skiprows += 1
print('skiprows =', skiprows)

# Read the CSV file.
names = []
for i in range(1, ndim + 1):
    names.extend([f'start{i}', f'stop{i}', f'step{i}'])
df = pd.read_table(csv_filename, skiprows=skiprows, sep='\s+', header=None, names=names)
df

Make sure the actuator names in this notebook match the order in the CSV file.

In [ ]:
header_line = None
for row, line in enumerate(open(join(datadir, filename + '.csv'), 'r')):
    if row == skiprows - 1:
        header_line = line
        break
for char in ['#', ',']:
    header_line = header_line.replace(char, '')
header_line = header_line.lstrip()
print(header_line.split())

In [ ]:
# Extract actuator points.
act_pts = [df.loc[:, [f'start{i}', f'stop{i}']].values.ravel() for i in range(1, ndim + 1)]
act_pts = np.array(act_pts).T

# Undo linear transformation.
act_pts_n = np.apply_along_axis(lambda pt: np.matmul(Minv, pt - center), 1, act_pts)

# Plot actuator points in "normalized" space.
fig, axes = pplt.subplots(nrows=3, figsize=(8.0, 4.0), spany=False, aligny=True)
for i, ax in enumerate(axes):
    ax.plot(act_pts_n[:, i], color='black', lw=None, marker='.', ms=0)
    ax.format(ylabel=f'actuator {i}')

Create dataframe of planned actuator points in real and normalized space.

In [ ]:
dim_names = ['y1', 'x2', 'x1']
columns = []
for i, dim_name in enumerate(dim_names):
    columns.extend([dim_name, dim_name + '_n'])
df = pd.DataFrame(index=df.index, columns=columns)
for i, dim_name in enumerate(dim_names):
    df[dim_name] = act_pts[::2, i]
    df[dim_name + '_n'] = act_pts_n[::2, i]
df

## Interpolation 

### Snap points onto x1-x2 grid

In [ ]:
GV, BI = [], []
for i, dim_name in enumerate(dim_names):
    gv, idx = utils.snap(act_pts_n[:, i], n=11*15)
    GV.append(gv)
    BI.append(idx)
    print(f'{dim_name}: {len(gv)} grid points')

In [ ]:
GV_X2_n, GV_X1_n = np.meshgrid(GV[1], GV[2], indexing='ij')

fig, ax = pplt.subplots(figwidth=4.0)
kws = dict(color='black', alpha=0.15)
for gv in GV[1]:
    ax.axvline(gv, **kws)
for gv in GV[2]:
    h1 = ax.axhline(gv, **kws)
h2 = ax.scatter(act_pts_n[:, 1], act_pts_n[:, 2], c='black', s=4, ec='None') 
ax.legend([h1, h2], labels=['grid', 'planned points'], loc='t')
ax.format(xlabel=dim_names[1], ylabel=dim_names[2])
plt.show()

`POINTS2D` holds the iteration number on the x2-x1 grid. (The number of iterations is half the number of sweeps.)

In [ ]:
POINTS2D = np.zeros((len(GV[2]), len(GV[1])))
POINTS2D[:] = np.nan
steps = []
iteration = 0
for i in range(POINTS2D.shape[0]):
    for j in range(POINTS2D.shape[1]):
        idx, = np.where((BI[2] == i) & (BI[1] == j))
        if len(idx) > 0:
            steps.append(idx)
            POINTS2D[i, j] = iteration
        else:
            print(f'Bin {i},{j} is empty')
        iteration += 1
POINTS2D = POINTS2D.astype(int)

The entry `idx_bin[i]` holds a list of all indices for iteration `i`.

In [ ]:
idx_bin = []
for i, step in enumerate(steps):
    indices = np.unique((step / 2 + 1).astype(int))
    idx = np.hstack([np.where(data_sc[:, 'iteration'] == j)[0] for j in indices])
    idx_bin.append(np.unique(idx))

### Observe one sweep

We will observe the sweep containing the largest camera integral in the scan.

In [ ]:
iteration_peak, n_iterations = None, len(steps)
for iteration in range(n_iterations):
    if ipeak in idx_bin[iteration]:
        iteration_peak = iteration
        
iteration = iteration_peak
idx = idx_bin[iteration]
print(f'Peak is in iteration {iteration}')
print(f'Set iteration = {iteration}')
print(f'Set idx = idx_bin[iteration] = {idx}')

Observe the actuator positions and camera integral during the sweep.

In [ ]:
bcm_mean = np.mean(data_sc[:, 'bcm04'])
bcm_scale = bcm_mean / data_sc[idx, 'bcm04']

kws = dict(marker='.', color='black')
fig, axes = pplt.subplots(nrows=4, figsize=(5, 6), spany=False, aligny=True)
for _scale, alpha, label in zip([1.0, bcm_scale], [1.0, 0.2], [None, 'Scaled by BCM04']):
    axes[0].plot(
        idx,
        data_sc[idx, cam + '_Integral'] * _scale,
        alpha=alpha,
        label=label,
        **kws,
    )
axes[0].legend(loc='upper right')
axes[0].format(ylabel=f'{cam}_Integral')
for ax, act in zip(axes[1:], acts):
    ax.plot(
        idx,
        data_sc[idx, act],
        **kws,
    )
    ax.format(ylabel=act)
axes.format(xlabel='Step in scan', suptitle=f'Iteration {iteration}')
plt.show()

Observe the image on the screen during the sweep. 

In [ ]:
def get_image(i):
    return data_im[i, cam + '_Image'].reshape(ny, nx)

In [ ]:
ipeak_sweep = np.argmax(data_sc[idx, cam + '_Integral'])
norm_pixel_value = np.max(data_im[idx[ipeak_sweep], cam + '_Image'])
cmap = pplt.Colormap('dusk_r')
greyscale = False

_ims = []
for i in tqdm(idx):
    _im = get_image(i) / norm_pixel_value
    if not greyscale:
        _im = cmap(_im)
    _ims.append(np.uint8(_im * np.iinfo(np.uint8).max))

gif_filename = f'_output/iteration{iteration}.gif'
imageio.mimwrite(gif_filename, _ims, fps=6)

![](_output/iteration742.gif)

### Smooth and interpolate along y_PositionSync 

We have snapped all points onto an x1-x2 grid. We now interpolate the y1 coordinate — the sweeping variable.

In [ ]:
# Assemble the 3d image.
im3d = np.array([get_image(i) for i in idx])

# Apply a smoothing filter along the first dimension (y_PositionSync). 
im3d_smooth = ndimage.median_filter(im3d, size=(3, 1, 1), mode='constant', cval=0.0) 

In [ ]:
i, _, _ = np.unravel_index(np.argmax(im3d), im3d.shape)

fig, axes = pplt.subplots(ncols=3, figwidth=7, sharey=False)
axes[0].pcolormesh(im3d[i, :, :])
axes[1].pcolormesh(im3d_smooth[i, :, :])
axes[2].plot(np.sum(im3d[i, :, :], axis=0), color='black', 
             marker='.', ms=3, label='raw')
axes[2].plot(np.sum(im3d_smooth[i, :, :], axis=0), color='pink6', 
             marker='.', ms=3, lw=0.75, label='smoothed')
for ax, title in zip(axes, ['Raw', 'Smoothed along y', 'Profiles']):
    ax.set_title(title)
axes[:-1].format(xticks=[], yticks=[])
axes[-1].format(yscale='log')
axes[-1].legend(ncol=1, loc='upper right')
plt.show()

### Interpolate to y1 grid

In [ ]:
# Define the y1 grid using all points in the data set.
y1_gv = np.linspace(
    np.min(data_sc['y_PositionSync']),
    np.max(data_sc['y_PositionSync']),
    N_PTS_Y,
)

# Obtain the y1 values corresponding to each image in this sweep.
y1_vals = data_sc[idx, 'y_PositionSync']

# Interpolate along the y1 axis.
if SMOOTH:
    arr3d = ip.interp_along_axis(im3d_smooth, y1_vals, y1_gv, axis=0)
else:
    arr3d = ip.interp_along_axis(im3d, y1_vals, y1_gv, axis=0)

In [ ]:
row, col = utils.max_indices(get_image(ipeak))

fig, axes = pplt.subplots(ncols=2, sharey=False)
for i, ax in enumerate(axes):
    for x, arr, alpha, label in zip([y1_gv, y1_vals], [arr3d, im3d], [1.0, 0.2], ['interpolated', 'raw']):
        if i == 0:
            y = utils.project(arr, 0)
        else:
            y = arr[:, row, col]
        ax.plot(x, y, color='black', label=label, marker='.', alpha=alpha)
axes[1].legend(loc='upper right', ncol=1,)
axes.format(yscale='log', xlabel='y [mm]')
for ax, title in zip(axes, ['Sum over image', f'Pixel ({row}, {col})']):
    ax.format(title=title)
plt.show()

Find the frame with the maximum pixel.

In [ ]:
max1 = np.unravel_index(np.argmax(im3d), im3d.shape)[0]
max2 = 1 + np.unravel_index(np.argmax(arr3d), arr3d.shape)[0]  # why + 1?
row1, col1 = utils.max_indices(im3d[max1, :, :])
row2, col2 = utils.max_indices(arr3d[max2, :, :])
print(f'max1, row1, col1 = ({max1}, {row1}, {col1})')
print(f'max2, row2, col2 = ({max2}, {row2}, {col2})')

In [ ]:
im1, im2 = im3d[max1], arr3d[max2]
fig, axes = pplt.subplots(ncols=2, nrows=2, figwidth=None, sharex=False, sharey=False)
for col, _im in enumerate([im1, im2]):
    for row, norm in enumerate([None, 'log']):
        mplt.plot_image(_im.T, ax=axes[row, col], norm=norm, colorbar=True)
axes.format(leftlabels=['Normal scale', 'Log scale'],
            toplabels=['Raw image', 'Interpolated along y'])

In [ ]:
fig, axes = pplt.subplots(ncols=2, sharey=False, spanx=False, figsize=(8, 2))
axes[0].plot(np.sum(arr3d[max2, :, :], axis=0), color='black', alpha=0.2, label='interpolated')
axes[0].plot(np.sum(im3d[max1, :, :], axis=0), marker='.', ms=2, lw=0, color='black', label='raw')
axes[0].legend(ncol=1)
axes[0].format(yscale='log', title='ProfileX')
axes[1].plot(arr3d[-2, :, :].ravel(), marker='.', lw=0, color='black', ms=1, label='Raw')
axes[1].plot(arr3d[-2, :, :].ravel(), color='black', alpha=0.2, label='Interpolated')
axes[1].legend(loc='upper left', ncols=1)
axes[1].format(title='Pixels in edge frame', xlabel='Pixel number')

In [ ]:
fig, axes = pplt.subplots([[1, 2, 3], [1, 4, 5]], sharey=False, sharex=False)
axes[0].pcolormesh(im3d[max1, :, :])
kws = dict(color='white', lw=0.785, alpha=0.5)
axes[0].axvline(col1, **kws)
axes[0].axhline(row1, **kws)
kws = dict(marker='.', color='black', ms=2.0, lw=1.0)
for im, row, col, label in zip([arr3d[max2], im3d[max1]], [row2, row1], [col2, col1], ['interpolated', 'raw']):
    if label == 'raw':
        kws['alpha'] = 0.2
    elif label == 'interpolated':
        kws['alpha'] = 1.0
    for ax in axes[:, 1]:
        ax.plot(im[row, :] / np.sum(im[row, :]), label=label, **kws)
    for ax in axes[:, 2]:
        ax.plot(im[:, col] / np.sum(im[:, col]), label=label, **kws)
axes[2].legend(ncols=1, loc='upper right')
axes[1].set_title(f'{cam} - row {row}')
axes[2].set_title(f'{cam} - col {col}')
axes[1, 1:].format(yscale='log')
plt.show()

## View projections and slices of interpolated 3D image 

### Projections

In [ ]:
dims = ['y1', 'x3', 'y3']
axes = mplt.corner(arr3d, diag_kind=None, labels=dims, prof='edges')

### Slices

In [ ]:
ind_slice = [int(s * 1 / 2) for s in arr3d.shape]

plot_kws = dict(discrete=False, colorbar=True)
fig, axes = pplt.subplots(nrows=2, ncols=3, sharex=False, sharey=False, spany=False, spanx=False)
for j in range(3):
    _im = utils.slice_array(arr3d, axis=j, ind=ind_slice[j])
    _im = _im / np.max(_im)
    for ax, norm in zip(axes[:, j], [None, 'log']):
        mplt.plot_image(_im, ax=ax, norm=norm, **plot_kws)
    axes[0, j].format(title=f'{dims[i]} = {ind_slice[i]}')
axes[:, 0].format(xlabel='y3', ylabel='x3')
axes[:, 1].format(xlabel='y1', ylabel='x3')
axes[:, 2].format(xlabel='y1', ylabel='y3')
plt.show()

## Save grid coordinates

In [ ]:
x2_x1_n = np.vstack([GV_X2_n.ravel(), GV_X1_n.ravel()]).T
x2_x1 = utils.apply(M[1:, 1:], x2_x1_n) + center[1:]
GV_X2 = x2_x1[:, 0].reshape(GV_X2_n.shape)
GV_X1 = x2_x1[:, 1].reshape(GV_X1_n.shape)

fig, axes = pplt.subplots(ncols=2, figwidth=8, sharey=False, sharex=False)
for ax, (grid1, grid2), pts in zip(axes, [(GV_X2, GV_X1), (GV_X2_n, GV_X1_n)], [act_pts, act_pts_n]):
    ax.plot(grid1[:], grid2[:], color='black', alpha=0.2, lw=3)
    ax.plot(pts[:, 1], pts[:, 2], color='black', marker='.', ms=2, lw=0.5)
axes.format(xlabel='x2', ylabel='x1')
axes[1].format(title='"Normalized"')

Obtain 3D grid coordinates.

In [ ]:
X1 = np.repeat(GV_X1[:, :, np.newaxis], N_PTS_Y, axis=2)
X2 = np.repeat(GV_X2[:, :, np.newaxis], N_PTS_Y, axis=2)
Y1 = np.zeros(X1.shape)
for i in range(len(GV[2])):
    for j in range(len(GV[1])):
        Y1[i, j, :] = y1_gv

In [ ]:
fig, axes = pplt.subplots(nrows=3, figsize=(5, 4), spany=False, aligny=True)
for ax, G, ylabel in zip(axes, [X1, X2, Y1], ["x1 [mm]", "x2 [mm]", "y1 [mm]"]):
    ax.plot(G.ravel(), color='black')
    ax.format(ylabel=ylabel)
axes.format(xlabel='Step')

In [ ]:
coord_3d = np.stack([X1, X2, Y1], axis=0)
print(f'coord_3d.shape = {coord_3d.shape}')

In [ ]:
savefilename = f'slit_coordinates_{filename}.npy'
np.save(savefilename, coord_3d)

## Save 5D array as memory map 

In [ ]:
shape = [len(GV[2]), len(GV[1])] + list(arr3d.shape)  # [x1, x2, y1, y3, x3]
shape = tuple(shape)
np.savetxt('rawgrid_shape.txt', shape)
print('shape:', shape)

In [ ]:
im_dtype = get_image(0).dtype
file = open('im_dtype.txt', 'w')
file.write(str(im_dtype))
file.close()
print('image dtype:', im_dtype)

In [ ]:
savefilename = f'rawgrid_{filename}.mmp'
arrays_3d = np.memmap(savefilename, shape=shape, dtype=im_dtype, mode='w+') 
for i in trange(POINTS2D.shape[0]):
    for j in range(POINTS2D.shape[1]):
        try:
            iteration = POINTS2D[i, j]
            idx = idx_bin[iteration]
            im3d = np.array([get_image(k) for k in idx])
            if SMOOTH:
                im3d = ndimage.median_filter(im3d, size=(3, 1, 1), mode='constant', cval=0.0) 
            y1_vals = data_sc[idx, 'y_PositionSync']
            arrays_3d[i, j, :, :, :] = ip.interp_along_axis(im3d, y1_vals, y1_gv, axis=0)
        except IndexError:
            print(f'No points in grid bin ({i}, {j})')
            arrays_3d[i, j, :, :, :] = 0
del arrays_3d